# **1. Perkenalan Dataset**


Tahap pertama, Anda harus mencari dan menggunakan dataset dengan ketentuan sebagai berikut:

1. **Sumber Dataset**:  
   Dataset dapat diperoleh dari berbagai sumber, seperti public repositories (*Kaggle*, *UCI ML Repository*, *Open Data*) atau data primer yang Anda kumpulkan sendiri.


Link kaggle dataset :https://www.kaggle.com/datasets/carrie1/ecommerce-data/data

# **2. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning atau deep learning.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# **3. Memuat Dataset**

Pada tahap ini, Anda perlu memuat dataset ke dalam notebook. Jika dataset dalam format CSV, Anda bisa menggunakan pustaka pandas untuk membacanya. Pastikan untuk mengecek beberapa baris awal dataset untuk memahami strukturnya dan memastikan data telah dimuat dengan benar.

Jika dataset berada di Google Drive, pastikan Anda menghubungkan Google Drive ke Colab terlebih dahulu. Setelah dataset berhasil dimuat, langkah berikutnya adalah memeriksa kesesuaian data dan siap untuk dianalisis lebih lanjut.

Jika dataset berupa unstructured data, silakan sesuaikan dengan format seperti kelas Machine Learning Pengembangan atau Machine Learning Terapan

In [ ]:
# Load data user
url='https://drive.google.com/uc?id=1l4OlQwoikH_mqFRJUZeJO64_pyr82oKg'
df = pd.read_csv(url, encoding='ISO-8859-1')
df.head()

# **4. Exploratory Data Analysis (EDA)**

Pada tahap ini, Anda akan melakukan **Exploratory Data Analysis (EDA)** untuk memahami karakteristik dataset.

Tujuan dari EDA adalah untuk memperoleh wawasan awal yang mendalam mengenai data dan menentukan langkah selanjutnya dalam analisis atau pemodelan.

In [ ]:
print("Info Dataset:")
df.info()

print("\nStatistik Deskriptif:")
df.describe()

# **5. Data Preprocessing**

Pada tahap ini, data preprocessing adalah langkah penting untuk memastikan kualitas data sebelum digunakan dalam model machine learning.

Jika Anda menggunakan data teks, data mentah sering kali mengandung nilai kosong, duplikasi, atau rentang nilai yang tidak konsisten, yang dapat memengaruhi kinerja model. Oleh karena itu, proses ini bertujuan untuk membersihkan dan mempersiapkan data agar analisis berjalan optimal.

Berikut adalah tahapan-tahapan yang bisa dilakukan, tetapi **tidak terbatas** pada:
1. Menghapus atau Menangani Data Kosong (Missing Values)
2. Menghapus Data Duplikat
3. Normalisasi atau Standarisasi Fitur
4. Deteksi dan Penanganan Outlier
5. Encoding Data Kategorikal
6. Binning (Pengelompokan Data)

Cukup sesuaikan dengan karakteristik data yang kamu gunakan yah. Khususnya ketika kami menggunakan data tidak terstruktur.

In [ ]:
df_clean = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)].copy()
df_clean = df_clean[df_clean['Quantity'] < 10000]
df_clean.dropna(subset=['CustomerID'], inplace=True)
df_clean = df_clean.drop_duplicates()
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

print(f"Ukuran data setelah cleaning: {df_clean.shape}")

In [ ]:
df_clean['Month_Year'] = df_clean['InvoiceDate'].dt.to_period('M')

monthly_sales = df_clean.pivot_table(
    index='StockCode',
    columns='Month_Year',
    values='Quantity',
    aggfunc='sum',
    fill_value=0
)

product_features = pd.DataFrame()
product_features['Avg_Sales'] = monthly_sales.mean(axis=1)
product_features['Std_Dev'] = monthly_sales.std(axis=1)
product_features['Max_Sales'] = monthly_sales.max(axis=1)

product_features['CV'] = product_features['Std_Dev'] / product_features['Avg_Sales']
product_features['CV'] = product_features['CV'].fillna(0)

product_info = df_clean.groupby('StockCode').agg({'UnitPrice': 'mean', 'Description': 'first'})
final_data = product_features.join(product_info)
final_data['Avg_Revenue'] = final_data['Avg_Sales'] * final_data['UnitPrice']

In [ ]:
final_data.reset_index().to_csv('data_training_final.csv', index=False)

In [ ]:
final_data.head()